# checkDropOffInsert
Input: URID, busRun(which has been updated after checkPickupInsert), insertionPoint (which is an ETA)
Output: new busRun schedule, which reflects the domino effect from both insertions (pickup and drop off) 
What it does: Looking only after the insertion point, tries to see if they can dropoff the new request in between two consecutive scheduled stops within the dropoff time window

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#helper function to insert the pickup row into the busRun dataframe
def insert_row(frame,new,row):
    top = frame[0:row]
    bottom = frame[row:]
    return pd.concat((top,new,bottom))

In [ ]:

def checkDropOffInsert(URID, busRun, insertionPoint):
    #code
    #URID is a dictionary, and busRun is a dataframe, insertionPoint is an ETA
    #Go through the busRun schedule, can I change this to just go through the correct ETAs and get rid of next if?
    for i in range(0, busRun.shape[0]-1):
        #only consider times after the pickup
        busRunSub = busRun.loc[busRun.ETA > insertionPoint,:]
        #only consider times within the drop off window
        busRunSub = busRunSub[URID['timeWindowDropOffStart'] < busRun.ETA & busRun.ETA < URID['timeWindowDropOffEnd']]
        if busRun.ETA.iloc[i] >insertionPoint:
            #look to see if can get from sched location to dropoff point in allowed time window
            allowedTime = busRun.ETA.iloc[i+1]-busRun.ETA.iloc[i]
            #check with routing function API once made
            timeToDropOff = getRoutingTime(busRun.Lat.iloc[i], busRun.Lon.iloc[i], URID['latDropoff'], URID['lonDropoff'])
            timeToSched = getRoutingTime(URID['latDropoff'], URID['lonDropoff'], busRun.Lat.iloc[i+1], busRun.Lon.iloc[i+1])
            totalTime = timeToDropOff + timeToSched
            urETA = busRun.ETA.iloc[i]+timeToDropOff
            if totalTime < allowedTime:
                #try inserting 
                #busRun = SchedTime,ETA, Lat, Lon, BookingId/ClientId, SpaceOn, SpaceOff, Activity, MobiAidCode 
                urRow= pd.DataFrame(np.array[[ , urETA, URID['latDropoff'], URID['lonDropoff'], URID['ClientId'], , URID['capacityConstraint']
                                             0, URID['MobiAidCode']]], columns=busRun.columns) #how do I put in NA for SpaceOff?
                busRun = insert_row(busRun, urRow, i)
            #need to reestimate all the ETAs after the dropOff point 
                
 
    return 
